# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп».  
Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.  
Обоснуйте корректность его работы.  

**Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.**  
Подбирать наилучшую модель не требуется.  

## Загрузка, анализ и подготовка данных

In [1]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


### Столбец `Возраст` 
Приведём к типу `int`

In [6]:
data['Возраст'] = data['Возраст'].astype('int')

### Столбец `Зарплата` 
Приведём к типу `int`

In [7]:
data['Зарплата'] = data['Зарплата'].astype('int')

### Обработка дубликатов

In [8]:
def show_duplicates(data):
    duplicates = data.duplicated().sum()
    print(f'Количество дубликатов: {duplicates} ({duplicates / len(data):.1%})')

In [9]:
show_duplicates(data)

Количество дубликатов: 153 (3.1%)


In [10]:
data = data.drop_duplicates().reset_index(drop=True)
show_duplicates(data)

Количество дубликатов: 0 (0.0%)


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 189.5 KB


### Формирование выборок

In [12]:
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

***
### Вывод

* В каждой строчке данных информация об одном заёмщике.
* Признаки заёмщиков: «Пол», «Возраст», «Зарплата», «Члены семьи», «Страховые выплаты».
* Привёл признаки «Возраст» и «Зарплата» к нужному типу.
* Удалил дубликаты (3.1%).
* Выборки сформированы.<br><br>
***

## Умножение матриц

### Постановка задачи

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $E$ — единичная матрица

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:**  
Признаки умножают на обратимую матрицу.  
Изменится ли качество линейной регрессии?  
Умножение на обратимую матрицу используется, чтобы зашифровать признаки.  

In [13]:
invertible_matrix = np.random.normal(size=(features.shape[1], features.shape[1]))
test_features = features @ invertible_matrix
test_features.describe()

,0,1,2,3
count,4847.000000,4847.000000,4847.000000,4847.000000
mean,24226.191309,-27937.145996,-64868.697622,-10028.584953
std,6059.532921,6990.387145,16218.677946,2507.099461
min,3209.297454,-55336.096952,-128458.365240,-19858.676750
25%,20154.391766,-32640.936962,-75775.880809,-11716.397240
50%,24409.041613,-28148.131960,-65360.947184,-10103.700754
75%,28302.260239,-23236.939725,-53977.101301,-8344.684671
max,47979.753787,-3696.459212,-8610.559812,-1332.947197


In [14]:
test_features.head()

,0,1,2,3
0,30117.553995,-34731.306305,-80645.384912,-12469.275052
1,23070.029551,-26598.158185,-61779.852679,-9551.348600
2,12748.386067,-14697.488487,-34139.568096,-5279.090643
3,25326.655650,-29208.151181,-67808.171712,-10480.159218
4,15845.061634,-18271.677624,-42432.731800,-6563.025622


Восстановить оригинальные данные без использования `invertible_matrix` невозможно.  
Осталось проверить, что предсказания не изменятся из-за шифрования.  

### Обоснование

#### Предсказания (исходные данные)

$$
a = Xw
$$
<br>
<center>где формула обучения</center>
$$
w = (X^T X)^{-1} X^T y
$$
<br>
<center>подставим $w$ в формулу $a$</center>
$$
a = X (X^T X)^{-1} X^T y
$$
<br>
<center>согласно свойствам обратной матрицы $ (X^T X)^{-1} = X^{-1} (X^T)^{-1} $</center>
<center>следовательно</center>
$$
a = X X^{-1} (X^T)^{-1} X^T y
$$
<br>
<center>согласно свойствам единичной матрицы $ X X^{-1} = E $</center>
<center>следовательно</center>
$$
a = E (X^T)^{-1} X^T y
$$
<br>
<center>согласно свойствам единичной матрицы $ (X^T)^{-1} X^T = E $</center>
<center>следовательно</center>
$$
a = E E y
$$
<br>
<center>так как $ E E = E $ и $ E y = y $</center>
<center>следовательно</center>
$$
a = y
$$

#### Предсказания (зашифрованные данные)

Аналогично предсказаниям на исходных данных

$$
a \scriptscriptstyle 1 \textstyle = XPw\scriptscriptstyle 1
$$
<br>
<center>где формула обучения</center>
$$
w\scriptscriptstyle 1 \textstyle= ((X P)^T (X P))^{-1} (X P)^T y
$$
<br>
<center>подставим $w\scriptscriptstyle 1$ в формулу $a\scriptscriptstyle 1$</center>
$$
a\scriptscriptstyle 1 \textstyle = (X P) ((X P)^T (X P))^{-1} (X P)^T y
$$
<br>
<center>согласно свойствам обратной матрицы $ ((X P)^T (X P))^{-1} = (X P)^{-1} ((X P)^T)^{-1} $</center>
<center>следовательно</center>
$$
a\scriptscriptstyle 1 \textstyle = (X P) (X P)^{-1} ((X P)^T)^{-1} (X P)^T y
$$
<br>
<center>согласно свойствам единичной матрицы $ (X P) (X P)^{-1} = E $</center>
<center>следовательно</center>
$$
a\scriptscriptstyle 1 \textstyle = E ((X P)^T)^{-1} (X P)^T y
$$
<br>
<center>согласно свойствам единичной матрицы $ ((X P)^T)^{-1} (X P)^T = E $</center>
<center>следовательно</center>
$$
a\scriptscriptstyle 1 \textstyle = E E y\scriptscriptstyle 1
$$
<br>
<center>так как $ E E = E $ и $ E y\scriptscriptstyle 1 \textstyle  = y $</center>
<center>следовательно</center>
$$
a\scriptscriptstyle 1 \textstyle = y
$$

**Обоснование:**

Используемые свойства:
$$ (AB)^T=B^T A^T $$
$$ (AB)^{-1} = B^{-1} A^{-1} $$
$$ A A^{-1} = A^{-1} A = E $$
$$ AE = EA = A $$
Доказать:
$$ a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w' $$
Доказательство:
$$ w = (X^T X)^{-1} X^T y $$

$$ w' = ((XP)^T XP)^{-1} (XP)^T y $$
$$ w' = (P^T (X^T X) P)^{-1} (XP)^T y $$
<div class="alert alert-info">
<center>Согласно свойству $ (AB)^{-1} = B^{-1} A^{-1} $</center>
$$ w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y $$
<br>
<center>Согласно свойству $ A A^{-1} = A^{-1} A = E $</center>
$$ w' = P^{-1} (X^T X)^{-1} E X^T y $$
<br>
<center>Согласно свойству $ AE = EA = A $ и так как $ w = (X^T X)^{-1} X^T y $</center>
$$ w' = P^{-1} w $$
<br>
<center>Согласно предположению выше,<br>
доказано, что $ a = a' $</center>
</div>

### Ответ

Из обоснования ясно, что предсказания не меняются при умножении матрицы признаков на обратимую матрицу.<br>
$$
a\scriptscriptstyle 1 \textstyle = a
$$

## Алгоритм преобразования

**Алгоритм**

Исходя из ответа прошлой задачи  
<i>«Изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу?»</i>  
Можно использовать умножение признаков на обратимую матрицу для того, чтобы защитить данные пользователей.  

**Обоснование**

Так как выше мы пришли к тому, что предсказания не изменятся на защищённых данных,  
то алгоритм является легитимным. 

## Проверка алгоритма

Проверка, что качество линейной регрессии из `sklearn` не отличается до и после преобразования с помощью метрики `R2`.

In [15]:
model = LinearRegression()

def show_r2_score(features, target):
    model.fit(features, target)
    predictions = model.predict(features)
    return r2_score(target, predictions)

In [16]:
r2_orginal = show_r2_score(features, target)
r2_test = show_r2_score(test_features, target)
r2_difference = r2_orginal - r2_test
print(
    f'Метрика R2 на исходных данных: {r2_orginal:0.2%}\n'
    f'Метрика R2 на защищённых данных: {r2_test:0.2%}\n'
    f'Разница метрик: {r2_difference * 1e13:0.3f} × 10⁻¹³'
)

Метрика R2 на исходных данных: 43.02%
Метрика R2 на защищённых данных: 43.02%
Разница метрик: 0.212 × 10⁻¹³


***
## Общий вывод

* Качество модели не меняется при защите данных путём умножения признаков на обратимую матрицу.
* Есть небольшая разница в показателях коэффициента детерминации (R2),  
    которую можно объяснить погрешностью при операциях с числами с плавающей точкой.<br><br>
***